# Insert Ids
This script opens a zone with seat data, and inserts the real identifiers from the backend.
What was previously called id, is renamed to seatNr.

In [13]:
import glob
import json
import requests

folders = glob.glob("zones/*/")
print(folders)

['zones\\1\\', 'zones\\10\\', 'zones\\11\\', 'zones\\12\\', 'zones\\13\\', 'zones\\14\\', 'zones\\15\\', 'zones\\16\\', 'zones\\17\\', 'zones\\18\\', 'zones\\19\\', 'zones\\2\\', 'zones\\20\\', 'zones\\21\\', 'zones\\22\\', 'zones\\23\\', 'zones\\24\\', 'zones\\25\\', 'zones\\27\\', 'zones\\28\\', 'zones\\29\\', 'zones\\3\\', 'zones\\33\\', 'zones\\34\\', 'zones\\35\\', 'zones\\36\\', 'zones\\37\\', 'zones\\38\\', 'zones\\39\\', 'zones\\4\\', 'zones\\40\\', 'zones\\41\\', 'zones\\42\\', 'zones\\43\\', 'zones\\45\\', 'zones\\46\\', 'zones\\48\\', 'zones\\49\\', 'zones\\5\\', 'zones\\50\\', 'zones\\51\\', 'zones\\52\\', 'zones\\59\\', 'zones\\62\\', 'zones\\63\\', 'zones\\64\\', 'zones\\65\\', 'zones\\8\\', 'zones\\9\\']


In [14]:
cookies = {}
with open("cookies.env", "r") as f:
    for entry in f.read().split("; "):
        cookies.update({entry.split("=")[0]: entry.split("=")[1]})

In [15]:
with open("zones/zones.json", 'r') as f:
    zones_data = json.load(f)

def getInfoByZone(zoneId: int):
    for city_data in zones_data:
        for library in city_data["libraries"]:
            for zone in library["zones"]:
                if zoneId == zone["id"]:
                    return {
                        "city": city_data["name"],
                        "library": library["name"],
                        "locationId": library["id"],
                        "zoneName": zone["name"]
                    }
    raise RuntimeError(f"No info found for {zoneId}.")

In [16]:
def fetchZone(zoneId: int):
    # fetch
    page = 0
    output_data = []
    while True:
        query = f"locationId={getInfoByZone(zoneId)['locationId']}&" + \
        f"zoneId={zoneId}&" + \
        f"resourceTypeId=302&" + \
        f"pageNumber={page}&" + \
        f"startDate=2025-09-05&" + \
        f"startTime=&" + \
        f"endDate=2025-09-05&" + \
        f"endTime=&" + \
        f"participantCount=1&" + \
        f"tagIds=&" + \
        f"exactMatch=true&" + \
        f"onlyFavorites=false&" + \
        f"resourceNameInfix=&version=1.0"
        # print(query)
        resp = requests.get(f"https://kurt3.ghum.kuleuven.be/api/resourcetypeavailabilities?"+query, cookies=cookies)
        data = resp.json()["availabilities"]
        output_data = output_data + data
        if len(data) == 60:
            page += 1
            continue
        return output_data

In [17]:
def convert_seat(availabilities, seat_data):
    seatNr = seat_data["seatNr"]
    # seat_data.update({"seatNr": old_id})
    seat_data.update({"id": None})
    availability_data = None
    for availability in availabilities:
        try:
            if int(availability["name"].split(" ")[-1]) == seatNr:
                # copy only few keys
                availability_data = {
                    "id": availability["id"],
                    "name": availability["name"],
                    "locationId": availability["locationId"],
                    "unit": availability["unit"],
                    "resourceTypeId": availability["resourceTypeId"],
                    "resourceTypeName": availability["resourceTypeName"],
                    "url": availability["url"],
                    "participantCount": availability["participantCount"],
                    "minParticipants": availability["minParticipants"],
                    "maxParticipants": availability["maxParticipants"],
                    "minDurationMinutes": availability["minDurationMinutes"],
                    "maxDurationMinutes": availability["maxDurationMinutes"],
                    "tags": availability["tags"],
                    "reservationRequestRequired": availability["reservationRequestRequired"],
                    "isMultiDayReservable": availability["isMultiDayReservable"],
                    "matchScore": availability["matchScore"],
                    # "otherScore": availability["otherScore"],
                    "fixedTimes": []
                }
        except ValueError:
            return seat_data
        # print(int(availability["name"].split(" ")[-1]))
        # print(old_id)
    if availability_data is None:
        return seat_data
    seat_data.update(availability_data)
    return seat_data

In [18]:
for folder in folders:
    zone_id = int(folder.split("\\")[1])
    output_data = {}
    with open(folder + "rectangles.json", 'r') as f:
        data =json.load(f)
    if type(data) is list:
        continue
    output_data["version"] = "2.0"
    output_data["image_width"] = data["image_width"]
    output_data["image_height"] = data["image_height"]
    output_data["seats"] = []
    try:
        output_data.update(getInfoByZone(zone_id))    
    except RuntimeError:
        # save_output(output_data)
        continue
    try:
        availabilities = fetchZone(zone_id)
    except json.JSONDecodeError:
        continue
    for seat_data in data["seats"]:
        output_data["seats"].append(convert_seat(availabilities, seat_data))
    print(output_data)
    # write result
    with open(folder + "rectangles.json", 'w+') as f:
        json.dump(output_data, f, indent=3)

{'version': '2.0', 'image_width': 1350, 'image_height': 1909, 'seats': [{'seatNr': 101, 'x': 1024, 'y': 1542, 'width': 45, 'height': 25, 'rotation': 0, 'id': 300854, 'name': 'Agora - Silent Study Seat 101', 'locationId': 10, 'unit': 'Agora Learning Centre', 'resourceTypeId': 302, 'resourceTypeName': 'Study Seat', 'url': 'https://kurt3.ghum.kuleuven.be/zone/1', 'participantCount': 1, 'minParticipants': 1, 'maxParticipants': 1, 'minDurationMinutes': 60, 'maxDurationMinutes': 960, 'tags': [{'id': 'power socket nearby', 'name': 'Power socket nearby', 'iconType': 0, 'icon': 'power'}, {'id': 'silence', 'name': 'Silence', 'iconType': 0, 'icon': 'voice_over_off'}], 'reservationRequestRequired': False, 'isMultiDayReservable': False, 'matchScore': 7, 'otherScore': 2, 'fixedTimes': []}, {'seatNr': 102, 'x': 1025, 'y': 1386, 'width': 45, 'height': 25, 'rotation': 0, 'id': 300855, 'name': 'Agora - Silent Study Seat 102', 'locationId': 10, 'unit': 'Agora Learning Centre', 'resourceTypeId': 302, 'res